# ***Clone*** ***The*** ***Repository***

In [ ]:
!nvidia-smi

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

# change makefile to have GPU and OPENCV enabled

In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

# DOWNLOAD PRE_TRAINED YOLOv4 WEIGHTS

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

***# GOOGLE-DRIVE CONNECTION.***
# **UPLOAD THE DATA ON GOOGLE-DRIVE** 

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# cd back into the darknet folder to run detections
%cd darknet

# ***Moving Your Custom Datasets Into Your Cloud VM***

In [ ]:
# this is where my datasets are stored within my Google Drive (I created a yolov4 folder to store all important files for custom training)
!ls /mydrive/Bag_Detection           #your custom folder name in google drive

In [ ]:
# copy over both datasets into the root directory of the Colab VM (comment out test.zip if you are not using a validation dataset)
!cp /mydrive/Bag_Detection/obj.zip ../
!cp /mydrive/Bag_Detection/test.zip ../

In [ ]:
# unzip the datasets and their contents so that they are now in /darknet/data/ folder
!unzip ../obj.zip -d data/
!unzip ../test.zip -d data/

# Configuring Files for Training



In [ ]:
# download cfg to google drive and change its name
!cp cfg/yolov4-custom.cfg /mydrive/Bag_Detection/Bag_Detection-obj.cfg

After executing the above step.

Now you need to edit the .cfg to fit your needs based on your object detector. Open it up in a code or text editor to do so.

If you downloaded cfg to google drive you can use the built in Text Editor by going to your google drive and double clicking on yolov4-obj.cfg and then clicking on the Open with drop down and selectin Text Editor.

I recommend having batch = 64 and subdivisions = 16 for ultimate results. If you run into any issues then up subdivisions to 32.

Make the rest of the changes to the cfg based on how many classes you are training your detector on.

Note: I set my max_batches = 6000, steps = 4800, 5400, I changed the classes = 1 in the three YOLO layers and filters = 18 in the three convolutional layers before the YOLO layers.

How to Configure Your Variables:

**width** = 416

**height** = 416 (these can be any multiple of 32, 416 is standard, you can sometimes improve results by making value larger like 608 but will slow down training)

max_batches = (# of classes) * 2000 (but no less than 6000 so if you are training for 1, 2, or 3 classes it will be 6000, however detector for 5 classes would have max_batches=10000)

steps = (80% of max_batches), (90% of max_batches) (so if your max_batches = 10000, then steps = 8000, 9000)

filters = (# of classes + 5) * 3 (so if you are training for one class then your filters = 18, but if you are training for 4 classes then your filters = 27)

Optional: If you run into memory issues or find the training taking a super long time. In each of the three yolo layers in the cfg, change one line from random = 1 to random = 0 to speed up training but slightly reduce accuracy of model. Will also help save memory if you run into any memory issues.

In [ ]:
# upload the custom .cfg back to cloud VM from Google Drive
!cp /mydrive/Bag_Detection/Bag_Detection-obj.cfg ./cfg

## **obj.names and obj.data**

In [ ]:
# upload the obj.names and obj.data files to cloud VM from Google Drive
!cp /mydrive/Bag_Detection/obj.names ./data    
!cp /mydrive/Bag_Detection/obj.data  ./data

# Generating train.txt and test.txt

In [ ]:
# upload the generate_train.py and generate_test.py script to cloud VM from Google Drive
!cp /mydrive/Bag_Detection/generate_train.py ./
!cp /mydrive/Bag_Detection/generate_test.py ./

In [ ]:
!python generate_train.py
!python generate_test.py

In [ ]:
# verify that the newly generated train.txt and test.txt can be seen in our darknet/data folder
!ls data/

# Download pre-trained weights for the convolutional layers

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

# Train Your Custom Object Detector!

# TIP: This training could take several hours depending on how many iterations you chose in the .cfg file. You will want to let this run as you sleep or go to work for the day, etc. However, Colab Cloud Service kicks you off it's VMs if you are idle for too long (30-90 mins).

# To avoid this hold (CTRL + SHIFT + i) at the same time to open up the inspector view on your browser.

# Paste the following code into your console window and hit Enter

In [ ]:
'''
function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)
'''

In [ ]:
# train your custom detector! (uncomment %%capture below if you run into memory issues or your Colab is crashing)
# %%capture
!./darknet detector train data/obj.data cfg/Bag_Detection-obj.cfg yolov4.conv.137 -dont_show -map

In [ ]:
# show chart.png of how custom object detector did with training
imShow('chart.png')

## TRICK: If for some reason you get an error or your Colab goes idle during training, you have not lost your partially trained model and weights! Every 100 iterations a weights file called yolov4-obj_last.weights is saved to mydrive/yolov4/backup/ folder (wherever your backup folder is). This is why we created this folder in our Google drive and not on the cloud VM. If your runtime crashes and your backup folder was in your cloud VM you would lose your weights and your training progress.

## We can kick off training from our last saved weights file so that we don't have to restart! WOOHOO! Just run the following command but with your backup location. 

In [ ]:
# kick off training from where it last saved
!./darknet detector train data/obj.data cfg/Bag_Detection-obj.cfg /mydrive/Bag_Detection/backup/Bag_Detection-obj_last.weights -dont_show

# Checking the Mean Average Precision (mAP) of Your Model

If you didn't run the training with the '-map- flag added then you can still find out the mAP of your model after training. Run the following command on any of the saved weights from the training to see the mAP value for that specific weight's file. I would suggest to run it on multiple of the saved weights to compare and find the weights with the highest mAP as that is the most accurate one!

NOTE: If you think your final weights file has overfitted then it is important to run these mAP commands to see if one of the previously saved weights is a more accurate model for your classes.

In [ ]:
!./darknet detector map data/obj.data cfg/Bag_Detection-obj.cfg /mydrive/Bag_Detection/backup/Bag_Detection-obj_1000.weights

# Run Your Custom Object Detector!!!

In [ ]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-obj.cfg
%cd ..

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights /mydrive/images/car2.jpg -thresh 0.3
imShow('predictions.jpg')